# Data Exploration & Feature Selection


**IBM Data Science Professional Certificate - Capstone Project**
<br><br/>
**Remington Oliver Sexton, Ph.D.**
<br><br/>
In this notebook we perform some exploratory data analysis to determine which features of our dataset will be good predictors of our target variables for our machine learning model. 
<br> <br/>
The data we are using for this project is compiled data from UK traffic accidents and vehicle information from the years 2005 through 2017, which can be accessed via [Kaggle](https://www.kaggle.com/tsiaras/uk-road-safety-accidents-and-vehicles?select=Accident_Information.csv).  The goal of this project is to develop a successful model that can predict the accident severity as a function of various predictors, such as road conditions and vehicle information.



In [ ]:
%matplotlib inline
# %matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sns
import folium # for mapping
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)

#### Import the Data


In [ ]:
df = pd.read_csv('../data/accidents_all.csv')#,low_memory=False)

Display the columns 

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

Let's get a list of the columns:

In [ ]:
cols = df.columns.tolist()
dtypes = df.dtypes.tolist()
# Print these into two columns
print('{0:<50}{1:<30}'.format('attribute', 'dtype'))
print('----------------------------------------------------------------')
for i in range(len(cols)):
    print('{0:<50}{1:<30}'.format(cols[i], str(dtypes[i])))

#### Drop Unnecessary Columns

Some of these columns do not contribute to *predicting* causes of accidents, but are descriptors of each incident or vehicle.  We drop the following columns:

| Attribute                                    | Reason                                       |
| -------------------------------------------- | -------------------------------------------- |
| Unnamed: 0                                   | N/A                                          |
| Did_Police_Officer_Attend_Scene_of_Accident  | N/A                                          |
| Local_Authority_(Highway)                    | *Local_Authority_(District)* for mapping.    |
| Location_Easting_OSGR                        | district is used for location mapping.       |
| Location_Northing_OSGR                       | district is used for location mapping.       |
| LSOA_of_Accident_Location                    | district is used for location mapping.       |
| Police_Force                                 | N/A                                          |
| InScotland                                   | N/A                                          |
| Vehicle_Reference                            | N/A                                          |
| model                                        | *make* used instead.                         |
| Year_x                                       | *date* and *age_of_vehicle* used instead.    |
| Year_y                                       | *date* and *age_of_vehicle* used instead.    |
| 1st_Road_Class                               | N/A                                          | 
| 1st_Road_Number                              | N/A                                          | 
| 2nd_Road_Class                               | N/A                                          | 
| 2nd_Road_Number                              | N/A                                          | 

In [ ]:
drop_cols = ['Unnamed: 0','Did_Police_Officer_Attend_Scene_of_Accident',
             'Local_Authority_(Highway)','Location_Easting_OSGR',
             'Location_Northing_OSGR','LSOA_of_Accident_Location','Police_Force',
             'InScotland','Vehicle_Reference','model','Year_x','Year_y',
             '1st_Road_Number','1st_Road_Class','2nd_Road_Number','2nd_Road_Class'
            ]
df.drop(drop_cols,axis=1,inplace=True)
df.head()

#### Mapping Out Accidents in the UK

The dataset includes a `.geojson` file, which we can use to generate a map of accidents using the logitude and latitude coordinates for each accident.  First lets compile the districts and the number of accidents in each district into a separate DataFrame:

In [ ]:
# Get number of accidents by district and put them into a dataframe
uk_dist = df['Local_Authority_(District)'].value_counts().reset_index()
uk_dist.rename(columns={'index':'district','Local_Authority_(District)':'count'}, inplace=True)
uk_dist#.head()

In [ ]:
uk_dist.describe()

In [ ]:
# Get the average coordinates to center the map on
coords = [df['Latitude'].mean(),df['Longitude'].mean()]
f = folium.Figure(width=750, height=750)
uk_map = folium.Map(location=coords, zoom_start=7, min_zoom = 2).add_to(f)
uk_map
# uk_map.save('/Users/rem/IBM_data_science/9_Capstone/figures/uk_map.html')


In [ ]:
# Get the average coordinates to center the map on
coords = [df['Latitude'].mean(),df['Longitude'].mean()]
f = folium.Figure(width=750, height=750)
uk_map = folium.Map(location=coords, zoom_start=7, min_zoom = 2).add_to(f)

# Load geojson file
uk_geo = '../data/Local_Authority_Districts__December_2017__Boundaries_in_the_UK__WGS84_.geojson'

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(uk_dist['count'].min(),
                              uk_dist['count'].max(),
                              6, dtype=int)
print(threshold_scale)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# Create a choropleth map
uk_map.choropleth(
    geo_data=uk_geo,
    data=uk_dist,
    columns=['district', 'count'],
    key_on='feature.properties.lad17nm',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Frequency of Accidents in UK',
    reset=True
)
# uk_map.save('/Users/rem/IBM_data_science/9_Capstone/figures/all_accidents.html')

# Display map
uk_map

What about counts as a function of accident severity?  Where are the most and least severe accidents occurring?

In [ ]:
# Group the mean accident severity with district
df_group = df.groupby(['Accident_Severity', 'Local_Authority_(District)']).size().reset_index(name='count')
df_group.rename(columns={'Local_Authority_(District)':'district','Accident_Severity':'accident_severity'}, inplace=True)
df_group.head()

In [ ]:
df_pivot = df_group.pivot(index='district',columns='accident_severity').reset_index()
df_pivot = pd.DataFrame(df_pivot.values,columns=['district','fatal','serious','slight'])
df_pivot.head()

In [ ]:
f = folium.Figure(width=750, height=750)
severity_map = folium.Map(location=coords, zoom_start=7).add_to(f)

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_pivot['fatal'].min(),
                              df_pivot['fatal'].max(),
                              6, dtype=float)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# Create a choropleth map
severity_map.choropleth(
    geo_data=uk_geo,
    data=df_pivot,
    columns=['district','fatal'],
    key_on='feature.properties.lad17nm',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Frequency of Fatalities in UK',
    reset=True
)
severity_map.save('/Users/rem/IBM_data_science/9_Capstone/figures/fatal_map.html')

# Display map
severity_map

In [ ]:
f = folium.Figure(width=750, height=750)
severity_map = folium.Map(location=coords, zoom_start=7).add_to(f)

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_pivot['serious'].min(),
                              df_pivot['serious'].max(),
                              6, dtype=float)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# Create a choropleth map
severity_map.choropleth(
    geo_data=uk_geo,
    data=df_pivot,
    columns=['district','serious'],
    key_on='feature.properties.lad17nm',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Frequency of Serious Injuries in UK',
    reset=True
)
severity_map.save('/Users/rem/IBM_data_science/9_Capstone/figures/serious_map.html')

# Display map
severity_map

In [ ]:
f = folium.Figure(width=750, height=750)
severity_map = folium.Map(location=coords, zoom_start=7).add_to(f)

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(df_pivot['slight'].min(),
                              df_pivot['slight'].max(),
                              6, dtype=float)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# Create a choropleth map
severity_map.choropleth(
    geo_data=uk_geo,
    data=df_pivot,
    columns=['district','slight'],
    key_on='feature.properties.lad17nm',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Frequency of Slight Injuries in UK',
    reset=True
)
severity_map.save('/Users/rem/IBM_data_science/9_Capstone/figures/slight_map.html')
# Display map
severity_map

We can see from these maps that higher accident severities and occurrences are associated with urban centers, and lower accident severities and occurrences with more suburban or rural districts.  Surprisingly, London has relatively few accidents, and lower severity compared to say Birmingham. 

#### Timeline of Accidents

We can also visualize accidents as a function of time.  Let's get the number of accidents per unique date.

In [ ]:
# Get number of accidents by district and put them into a dataframe
df_date = df['Date'].value_counts().reset_index()
df_date.rename(columns={'index':'date','Date':'count'}, inplace=True)
df_date.sort_values(by='date')

We can now plot this data:

In [ ]:
# Convert dates to datetime objects
df_date['date'] = pd.to_datetime(df_date['date'])
df_date

In [ ]:
fig = plt.figure(figsize=(14,4))
ax1 = fig.add_subplot(1,1,1)

sns.lineplot(ax=ax1,data=df_date, x='date', y='count',
            linewidth=0.5)

fontsize=14
ax1.set_title('Timeline of Accidents in the UK (2005-2017)',fontsize=fontsize)
ax1.set_ylabel('Number of Accidents',fontsize=fontsize)
ax1.set_xlabel('Date',fontsize=fontsize)
ax1.set_xticks(['2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01',
                '2011-01-01','2012-01-01','2013-01-01','2014-01-01','2015-01-01','2016-01-01',
                '2017-01-01'])
ax1.set_xticklabels(['2005','2006','2007','2008','2009','2010',
                '2011','2012','2013','2014','2015','2016',
                '2017'])
ax1.tick_params(axis='both', labelsize=fontsize)
ax1.set_xlim('2005','2017')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/accidents_timeline.png',dpi=300)



It would seem that accidents are increasing over time, however the cause of this is likely outside the scope of this data.  Over this long period of time, there could be increased reporting due to better data collection, or simply an increase of drivers on the road. 

### Timeline by Accident Severity

In [ ]:
# Group the mean accident severity with date
df_group = df.groupby(['Accident_Severity', 'Date']).size().reset_index(name='count')
df_group.rename(columns={'Date':'date','Accident_Severity':'accident_severity'}, inplace=True)
# df_group['date'] = pd.to_datetime(df_date['date'])
df_group

In [ ]:
df_pivot = df_group.pivot(index='date',columns='accident_severity').reset_index()
df_pivot = pd.DataFrame(df_pivot.values,columns=['date','fatal','serious','slight'])
df_pivot.head()

In [ ]:
df_pivot['date'] = pd.to_datetime(df_pivot['date'])
df_pivot['fatal'].replace(np.nan,0.0,inplace=True)
# df_pivot['fatal'] = df_pivot['fatal']/df_pivot['fatal'].sum()
df_pivot['serious'].replace(np.nan,0.0,inplace=True)
# df_pivot['serious'] = df_pivot['serious']/df_pivot['serious'].sum()

df_pivot['slight'].replace(np.nan,0.0,inplace=True)
# df_pivot['slight'] = df_pivot['slight']/df_pivot['slight'].sum()

df_pivot['slight_%'] = df_pivot['slight']/df_pivot[['fatal','serious','slight']].sum(axis=1)*100.0
df_pivot['serious_%'] = df_pivot['serious']/df_pivot[['fatal','serious','slight']].sum(axis=1)*100.0
df_pivot['fatal_%'] = df_pivot['fatal']/df_pivot[['fatal','serious','slight']].sum(axis=1)*100.0

df_pivot

In [ ]:
fig = plt.figure(figsize=(14,4))
ax1 = fig.add_subplot(1,1,1)

sns.lineplot(ax=ax1,data=df_pivot, x='date', y='fatal_%',
            linewidth=0.5, color='xkcd:dark red', label='Fatal')
sns.lineplot(ax=ax1,data=df_pivot, x='date', y='serious_%',
            linewidth=0.5, color='xkcd:dark orange', label='Serious')
sns.lineplot(ax=ax1,data=df_pivot, x='date', y='slight_%',
            linewidth=0.5, color='xkcd:goldenrod', label='Slight')

fontsize=14
ax1.set_title('Timeline of Percent of Accidents by Date (2005-2017)',fontsize=fontsize)
ax1.set_ylabel('Percent of Accidents',fontsize=fontsize)
ax1.set_xlabel('Date',fontsize=fontsize)
ax1.set_xticks(['2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01',
                '2011-01-01','2012-01-01','2013-01-01','2014-01-01','2015-01-01','2016-01-01',
                '2017-01-01'])
ax1.set_xticklabels(['2005','2006','2007','2008','2009','2010',
                '2011','2012','2013','2014','2015','2016',
                '2017'])
ax1.tick_params(axis='both', labelsize=fontsize)
ax1.set_xlim('2005','2017')
ax1.legend()
plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/severity_timeline.png',dpi=300)



### Continuous Variable Feature Selection

Now we explore the data to determine which *continuous* features would be good predictors of accident severity.  There are only 5 continuous variables whose descriptions would be useful in the context of accident prevention.  In the below table, we list the continuous variables under consideration and their brief descriptions:

| Attribute                                    | 
| -------------------------------------------- | 
| Time | 
| Age_of_Vehicle | 
| Engine_Capacity_.CC. | 

The first thing we can do is create a correlation scatter matrix to visualize any linear relationships among these variables with accident severity, to see which of these are good predictors.  To do this, we first need to convert `Accident_Severity` from a categorical variable to a numerical one by label encoding (*note*: when we perform ML analysis, we must use one-hot encoding, not label encoding):

In [ ]:
# Extract continuous features
cont_attrib = ['Time','Age_of_Vehicle','Engine_Capacity_.CC.','Accident_Severity']
df_cont = df.copy()[cont_attrib]
df_cont.head()

In [ ]:
# Lets also conver time to a datatime format
hour = pd.to_datetime(df_cont['Time'],format= '%H:%M').dt.hour
minute = pd.to_datetime(df_cont['Time'],format= '%H:%M').dt.minute/60.0
df_cont['time'] = hour+minute
df_cont.head()

In [ ]:
fig = plt.figure(figsize=(14,4))
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

sns.boxplot(ax=ax1, x="Accident_Severity", y="Age_of_Vehicle", data=df_cont)
sns.boxplot(ax=ax2, x="Accident_Severity", y="Engine_Capacity_.CC.", data=df_cont)
sns.boxplot(ax=ax3, x="Accident_Severity", y="time", data=df_cont)

fontsize=14
# ax1.set_title('',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
ax2.set_xlabel('Accident Severity',fontsize=fontsize)
ax3.set_xlabel('Accident Severity',fontsize=fontsize)

ax1.set_ylabel('Age of Vehicle',fontsize=fontsize)
ax2.set_ylabel('Engine Capacity (CC)',fontsize=fontsize)
ax3.set_ylabel('Time of Day (24h)',fontsize=fontsize)
ax3.set_yticks([0,6,12,18,24])
ax3.set_yticklabels(['00:00','06:00','12:00','18:00','24:00'])

ax1.tick_params(axis='both', labelsize=fontsize)
ax2.tick_params(axis='both', labelsize=fontsize)
ax3.tick_params(axis='both', labelsize=fontsize)

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/continuous_attrib.png',dpi=300)


We can see that there are significant outliers for the age and engine capacity as a function of accident severity, which means these are probably not good predictors.  However, there are no outliers for the time predictor, which indicates that there is at least a certain time of day when accident are more probable regardless of severity.  This means it isn't a good predictor of accident severity, but moreso a predictor of when accidents are more likely to occur.  We thus do not include any of these variables in our model for predicting accident severity.
<br><br/>
We can see this better by plotting each distribution of accident severity as a function of time:

In [ ]:
time_slight  = df_cont.loc[df_cont['Accident_Severity']=='Slight','time']
time_serious = df_cont.loc[df_cont['Accident_Severity']=='Serious','time']
time_fatal   = df_cont.loc[df_cont['Accident_Severity']=='Fatal','time']

In [ ]:
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(1,1,1)

sns.kdeplot(data=time_slight,ax=ax1,
            fill=True,palette='crest',alpha=0.25,
            label='slight',linewidth=0)
sns.kdeplot(data=time_serious,ax=ax1,
           fill=True,palette='crest',alpha=0.25,
           label='serious',linewidth=0)
sns.kdeplot(data=time_fatal,ax=ax1,
           fill=True,palette='crest',alpha=0.25,
           label='fatal',linewidth=0)
fontsize=14
ax1.set_ylim(0,0.11)
ax1.set_xlabel('Time',fontsize=fontsize)
ax1.set_ylabel('Accidents (Normalized Density)',fontsize=fontsize)
ax1.set_title('KDE of Accidents as a function of Time of Day',fontsize=fontsize)
ax1.set_xticks([0,2,4,6,8,10,12,14,16,18,20,22,24])
ax1.set_xticklabels(['12:00 AM','2:00 AM','4:00 AM','6:00 AM','8:00 AM','10:00 AM',
                     '12:00 PM','2:00 PM','4:00 PM','6:00 PM','8:00 PM','10:00 PM','12:00 AM'
                    ], rotation=45, horizontalalignment='right')

ax1.annotate('Morning Rush', xy=(0.39, 0.77),  xycoords='axes fraction',
            xytext=(0.39, 0.77), textcoords='axes fraction',
            horizontalalignment='center', verticalalignment='center',
            fontsize=fontsize)
ax1.annotate('Evening Rush', xy=(0.67, 0.90),  xycoords='axes fraction',
            xytext=(0.67, 0.90), textcoords='axes fraction',
            horizontalalignment='center', verticalalignment='center',
            fontsize=fontsize)
ax1.tick_params(axis='both', labelsize=fontsize)

ax1.legend(fontsize=fontsize)
plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/accidents_time_of_day.png',dpi=300)


#### Day of Week

We could also see if the day of the week is important.

In [ ]:
# Group the mean accident severity with district
df_group = df.groupby(['Accident_Severity', 'Day_of_Week']).size().reset_index(name='count')
df_group.head(20)

In [ ]:
df_pivot = df_group.pivot(index='Day_of_Week',columns='Accident_Severity').reset_index()
df_pivot = pd.DataFrame(df_pivot.values,columns=['Day_of_Week','fatal','serious','slight'])
df_pivot.head(10)

In [ ]:
# Normalize each day so it is a percentage
df_pivot['fatal'] = df_pivot['fatal']/df_pivot['fatal'].sum()*100.
df_pivot['serious'] = df_pivot['serious']/df_pivot['serious'].sum()*100.
df_pivot['slight'] = df_pivot['slight']/df_pivot['slight'].sum()*100.
# Set index to day of week
df_pivot = df_pivot.set_index(['Day_of_Week'])
df_pivot

In [ ]:
from matplotlib.colors import ListedColormap

categories = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(1,1,1)

df_pivot.reindex(categories).plot(kind='bar', stacked=True, figsize=(10, 5),
                            fontsize=12, rot=0,ax=ax1,
                                  colormap=ListedColormap(sns.color_palette("YlOrRd_r", 10)))

fontsize=14
ax1.set_title('Percentage of Accidents Per Day of Week',fontsize=fontsize)
ax1.set_ylabel('\% of Accidents',fontsize=fontsize)
ax1.set_xlabel('Day of Week',fontsize=fontsize)
ax1.set_xticks(range(len(categories)))
ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
ax1.legend(fontsize=fontsize-2)

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/accidents_day_of_week.png',dpi=300)


We can see that day of week also does not have a significant influence on accident severity.

### Feature Selection

#### Categorical Variables 

Most of our predictors are categorical.  Below is a list of the predictors we will investigate:

| Attribute                                    | 
| -------------------------------------------- | 
| Light_Conditions |
| Road_Surface_Conditions | 
| Road_Type | 
| Speed_limit | 
| Urban_or_Rural_Area |
| Weather_Conditions | 
| Age_Band_of_Driver | 
| Vehicle_Type |

In [ ]:
# print out the categories in each column 
columns = ['Light_Conditions','Road_Surface_Conditions','Road_Type','Speed_limit',
           'Urban_or_Rural_Area','Weather_Conditions','Age_Band_of_Driver','Vehicle_Type']

for c in columns:
    print('Column: %s' % c)
    print('-------------------------------------------------')
    print(df[c].value_counts())
    print('-------------------------------------------------')
    print()

 Let's make a copy of the dataframe that we can manipulate:

In [ ]:
df_cat = df.copy()[['Accident_Severity','Light_Conditions','Road_Surface_Conditions','Road_Type','Speed_limit',
           'Urban_or_Rural_Area','Weather_Conditions','Age_Band_of_Driver','Vehicle_Type']]
print(df_cat.shape)
df_cat.head(25)

### Remove missing Data

Fortunately, because of the large size of this dataset, we can afford to remove data for which there is no missing value.  In reality, trying to replace these categorical data with something else would probably cause more problems than solve, so we just remove missing or unknown data altogeher.

In [ ]:
# Remove missing/unknown data from Light_Conditions
df_cat.drop(df_cat[df_cat['Light_Conditions']=='Data missing or out of range'].index, inplace = True)
# Also remove unknown lighting conditions
df_cat.drop(df_cat[df_cat['Light_Conditions']=='Darkness - lighting unknown'].index, inplace = True) 
# Remove missing/unknown data from Road_Surface_Conditions
df_cat.drop(df_cat[df_cat['Road_Surface_Conditions']=='Data missing or out of range'].index, inplace = True) 
# Remove missing/unknown data from Road_Type
df_cat.drop(df_cat[df_cat['Road_Type']=='Data missing or out of range'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Road_Type']=='Unknown'].index, inplace = True) 
# Remove missing/unknown data from Urban_or_Rural_Area
df_cat.drop(df_cat[df_cat['Urban_or_Rural_Area']=='Unallocated'].index, inplace = True) 
# Remove missing/unknown data from Weather_Conditions
df_cat.drop(df_cat[df_cat['Weather_Conditions']=='Data missing or out of range'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Weather_Conditions']=='Unknown'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Weather_Conditions']=='Other'].index, inplace = True) 
# Remove missing/unknown data from Weather_Conditions
df_cat.drop(df_cat[df_cat['Age_Band_of_Driver']=='Data missing or out of range'].index, inplace = True) 
# Let's get rid of some non-sensical driver ages
# The legal driving age in the UK is 15 years, 9 months or ~16 years; any younger and its illegal anyway.  These illegal age bands 
# just add noise the the data, and aren't worth keeping.
df_cat.drop(df_cat[df_cat['Age_Band_of_Driver']=='11 - 15'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Age_Band_of_Driver']=='6 - 10'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Age_Band_of_Driver']=='0 - 5'].index, inplace = True) 
# Remove missing/unknown data, and uncommon means of transportation from Vehicle_Type
df_cat.drop(df_cat[df_cat['Vehicle_Type']=='Data missing or out of range'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Vehicle_Type']=='Other vehicle'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Vehicle_Type']=='Ridden horse'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Vehicle_Type']=='Tram'].index, inplace = True) 

print(df_cat.shape)

### Accident Severity & Light Conditions

We can use a heatmap to visualize the relationship between these two categorical attributes.

In [ ]:
df_cat['Light_Conditions'].value_counts()

In [ ]:
# The "Darkness - lights unlit" may as well be "Darkness - no lighting", so we combine these two categories
df_cat['Light_Conditions'].replace('Darkness - lights unlit','Darkness - no lighting',inplace=True)
df_cat['Light_Conditions'].value_counts()

In [ ]:
df_group = df_cat.groupby(['Accident_Severity', 'Light_Conditions']).size().reset_index(name='count')

df_pivot = pd.pivot_table(df_group, values='count', 
                     index=['Light_Conditions'], 
                     columns='Accident_Severity')
# Normalize each day so it is a percentage
df_pivot['Fatal'] = df_pivot['Fatal']/df_pivot['Fatal'].sum()*100.
df_pivot['Serious'] = df_pivot['Serious']/df_pivot['Serious'].sum()*100.
df_pivot['Slight'] = df_pivot['Slight']/df_pivot['Slight'].sum()*100.
df_pivot

In [ ]:
fig  = plt.figure(figsize=(7,7))
ax1 = fig.add_subplot(1,1,1)

sns.heatmap(df_pivot, annot=True,fmt='.3f', linewidths=.3, ax=ax1,  cmap='GnBu')

fontsize=14
ax1.set_title('Percentage of Accidents by Accident Severity: Light Conditions',fontsize=fontsize-2)
ax1.set_ylabel('Light Conditions',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
# ax1.set_xticks(range(len(categories)))
# ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
plt.yticks(va='center')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/light_conditions.png',dpi=300)


It would seem that when there is lights lit or daylight, accident occurence is nearly the same across severities, but severity dramatically increases when there is no lighting.

### Accident Severity & Road Surface Conditions

In [ ]:
df_cat['Road_Surface_Conditions'].value_counts()

In [ ]:
df_cat['Road_Surface_Conditions'].replace('Wet or damp','Wet',inplace=True)
df_cat['Road_Surface_Conditions'].replace('Frost or ice','Ice',inplace=True)
df_cat['Road_Surface_Conditions'].replace('Flood over 3cm. deep','Flood',inplace=True)


In [ ]:
df_cat['Road_Surface_Conditions'].value_counts()

In [ ]:
df_group = df_cat.groupby(['Accident_Severity', 'Road_Surface_Conditions']).size().reset_index(name='count')

df_pivot = pd.pivot_table(df_group, values='count', 
                     index=['Road_Surface_Conditions'], 
                     columns='Accident_Severity')
# Normalize each day so it is a percentage
df_pivot['Fatal'] = df_pivot['Fatal']/df_pivot['Fatal'].sum()*100.
df_pivot['Serious'] = df_pivot['Serious']/df_pivot['Serious'].sum()*100.
df_pivot['Slight'] = df_pivot['Slight']/df_pivot['Slight'].sum()*100.
df_pivot

In [ ]:
fig  = plt.figure(figsize=(7,7))
ax1 = fig.add_subplot(1,1,1)
sns.heatmap(df_pivot, annot=True,fmt='.3f', linewidths=.3, ax=ax1,  cmap='YlOrRd')

fontsize=14
ax1.set_title('Percentage of Accidents by Accident Severity: Road Surface Conditions',fontsize=fontsize-2)
ax1.set_ylabel('Road Surface Conditions',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
# ax1.set_xticks(range(len(categories)))
# ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
plt.yticks(va='center')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/road_surface_cond.png',dpi=300)


Here we can see that accident severity is nearly constant across all road surface conditions, and therefore may be a bad predictor of severity.  This can be driven by factors we cannot account for with this data alone, such as the fact that drivers probably drive more cautiously during non-dry conditions. 

### Accident Severity & Road Type

In [ ]:
df_cat['Road_Type'].value_counts()

In [ ]:
df_cat['Road_Type'].replace('Slip road','On/off ramp',inplace=True)
df_cat['Road_Type'].value_counts()

In [ ]:
df_group = df_cat.groupby(['Accident_Severity', 'Road_Type']).size().reset_index(name='count')

df_pivot = pd.pivot_table(df_group, values='count', 
                     index=['Road_Type'], 
                     columns='Accident_Severity')
# Normalize each day so it is a percentage
df_pivot['Fatal'] = df_pivot['Fatal']/df_pivot['Fatal'].sum()*100.
df_pivot['Serious'] = df_pivot['Serious']/df_pivot['Serious'].sum()*100.
df_pivot['Slight'] = df_pivot['Slight']/df_pivot['Slight'].sum()*100.
df_pivot

In [ ]:
fig  = plt.figure(figsize=(7,10))
ax1 = fig.add_subplot(1,1,1)
sns.heatmap(df_pivot, annot=True,fmt='.3f', linewidths=.3, ax=ax1,  cmap='YlGnBu')

fontsize=14
ax1.set_title('Percentage of Accidents by Accident Severity: Road Type',fontsize=fontsize-1)
ax1.set_ylabel('Road Type',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
# ax1.set_xticks(range(len(categories)))
# ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
plt.yticks(va='center')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/road_type.png',dpi=300)


Again, not many good predictors fo severity here.  The only one that stands out is roundabouts, where fatal accidents decrease sharpy from slight accidents.

### Accident Severity & Speed Limit

In [ ]:
df_cat['Speed_limit'].value_counts()

Let's combine some of these labels to just two: "Under 40" and "Over 40".  We use 40 km/s (25 mph) because this is a typical resident speed limit, whereas anything over 40 km/s is closer to highway speed limit.

In [ ]:
df_cat['Speed_limit'].replace(15.0,'Under 40 km/s',inplace=True)
df_cat['Speed_limit'].replace(10.0,'Under 40 km/s',inplace=True)
df_cat['Speed_limit'].replace(20.0,'Under 40 km/s',inplace=True)
df_cat['Speed_limit'].replace(30.0,'Under 40 km/s',inplace=True)
df_cat['Speed_limit'].replace(40.0,'Under 40 km/s',inplace=True)
df_cat['Speed_limit'].replace(50.0,'Over 40 km/s',inplace=True)
df_cat['Speed_limit'].replace(60.0,'Over 40 km/s',inplace=True)
df_cat['Speed_limit'].replace(70.0,'Over 40 km/s',inplace=True)


In [ ]:
df_cat['Speed_limit'].value_counts()

In [ ]:
df_group = df_cat.groupby(['Accident_Severity', 'Speed_limit']).size().reset_index(name='count')

df_pivot = df_group.pivot(index='Speed_limit',columns='Accident_Severity').reset_index()
df_pivot = pd.DataFrame(df_pivot.values,columns=['Speed_limit','fatal','serious','slight'])

# Normalize each day so it is a percentage
df_pivot['fatal'] = df_pivot['fatal']/df_pivot['fatal'].sum()*100.
df_pivot['serious'] = df_pivot['serious']/df_pivot['serious'].sum()*100.
df_pivot['slight'] = df_pivot['slight']/df_pivot['slight'].sum()*100.
# Get rid of NaN
df_pivot.replace(np.nan, 0.0, inplace = True)

# Set index to day of week
df_pivot = df_pivot.set_index(['Speed_limit'])
df_pivot

In [ ]:

fig  = plt.figure(figsize=(7,4))
ax1 = fig.add_subplot(1,1,1)

sns.heatmap(df_pivot, annot=True,fmt='.3f', linewidths=.3, ax=ax1,  cmap='PuBuGn')

fontsize=14
ax1.set_title('Percentage of Accidents by Accident Severity: Posted Speed Limit',fontsize=fontsize-2)
ax1.set_ylabel('Posted Speed Limit',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
# ax1.set_xticks(range(len(categories)))
# ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
plt.yticks(va='center')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/speed_limit.png',dpi=300)


It would seem that fatalities increase dramatically when travelling at 60 km/hr, compared to other speeds, and slight accidents increase dramatically at 30 km/hr.

### Accident Severity & Urban or Rural Area

In [ ]:
df_cat['Urban_or_Rural_Area'].value_counts()

In [ ]:
df_group = df_cat.groupby(['Accident_Severity', 'Urban_or_Rural_Area']).size().reset_index(name='count')

df_pivot = df_group.pivot(index='Urban_or_Rural_Area',columns='Accident_Severity').reset_index()
df_pivot = pd.DataFrame(df_pivot.values,columns=['Urban_or_Rural_Area','fatal','serious','slight'])

# Normalize each day so it is a percentage
df_pivot['fatal'] = df_pivot['fatal']/df_pivot['fatal'].sum()*100.
df_pivot['serious'] = df_pivot['serious']/df_pivot['serious'].sum()*100.
df_pivot['slight'] = df_pivot['slight']/df_pivot['slight'].sum()*100.
# Get rid of NaN
df_pivot.replace(np.nan, 0.0, inplace = True)

# Set index to day of week
df_pivot = df_pivot.set_index(['Urban_or_Rural_Area'])
df_pivot

In [ ]:
categories = ['Urban','Rural']

fig  = plt.figure(figsize=(7,4))
ax1 = fig.add_subplot(1,1,1)

# df_pivot.plot(kind='bar', stacked=True, figsize=(10, 5),
#                             fontsize=12, rot=0,
#                                   colormap=ListedColormap(sns.color_palette("RdPu", 10)))

sns.heatmap(df_pivot, annot=True,fmt='.3f', linewidths=.3, ax=ax1,  cmap='BuGn')

fontsize=14
ax1.set_title('Percentage of Accidents by Accident Severity: Urban or Rural',fontsize=fontsize-2)
ax1.set_ylabel('Urban or Rural',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
# ax1.set_xticks(range(len(categories)))
# ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
plt.yticks(va='center')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/urban_rural.png',dpi=300)


Also interestingly, there are more fatalities for rural areas than uban ones. 

### Accident Severity & Age Range

In [ ]:
df_cat['Age_Band_of_Driver'].value_counts()

Let's combine features into "Adolescient" (16-20), "Adult" (21-65), and "Senior" (66-75+):

In [ ]:
df_cat['Age_Band_of_Driver'].replace('16 - 20','Adolescent',inplace=True)

df_cat['Age_Band_of_Driver'].replace('21 - 25','Adult',inplace=True)
df_cat['Age_Band_of_Driver'].replace('26 - 35','Adult',inplace=True)
df_cat['Age_Band_of_Driver'].replace('36 - 45','Adult',inplace=True)
df_cat['Age_Band_of_Driver'].replace('46 - 55','Adult',inplace=True)
df_cat['Age_Band_of_Driver'].replace('56 - 65','Adult',inplace=True)

df_cat['Age_Band_of_Driver'].replace('66 - 75','Senior',inplace=True)
df_cat['Age_Band_of_Driver'].replace('Over 75','Senior',inplace=True)


In [ ]:
df_group = df_cat.groupby(['Accident_Severity', 'Age_Band_of_Driver']).size().reset_index(name='count')

df_pivot = pd.pivot_table(df_group, values='count', 
                     index=['Age_Band_of_Driver'], 
                     columns='Accident_Severity')
# Normalize each day so it is a percentage
df_pivot['Fatal'] = df_pivot['Fatal']/df_pivot['Fatal'].sum()*100.
df_pivot['Serious'] = df_pivot['Serious']/df_pivot['Serious'].sum()*100.
df_pivot['Slight'] = df_pivot['Slight']/df_pivot['Slight'].sum()*100.
df_pivot

In [ ]:
fig  = plt.figure(figsize=(7,7))
ax1 = fig.add_subplot(1,1,1)
sns.heatmap(df_pivot, annot=True,fmt='.3f', linewidths=.3, ax=ax1,  cmap='BuGn')

fontsize=14
ax1.set_title('Percentage of Accidents by Accident Severity: Age Band of Driver',fontsize=fontsize-2)
ax1.set_ylabel('Age Band of Driver',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
# ax1.set_xticks(range(len(categories)))
# ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
plt.yticks(va='center')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/age_band.png',dpi=300)


There are no significant trends in Age Band with accident severity.  There is a slight increase for falat versus slight accidents for drivers over 75, but this is about it.

### Accident Severity & Weather Conditions

In [ ]:
df_cat['Weather_Conditions'].value_counts()

Let's combine some of these features to make things simpler:

In [ ]:
df_cat['Weather_Conditions'].replace('Fine no high winds','Fine',inplace=True)
df_cat['Weather_Conditions'].replace('Fine + high winds','Fine',inplace=True)
df_cat['Weather_Conditions'].replace('Raining no high winds','Rain',inplace=True)
df_cat['Weather_Conditions'].replace('Raining + high winds','Rain',inplace=True)
df_cat['Weather_Conditions'].replace('Snowing no high winds','Snow',inplace=True)
df_cat['Weather_Conditions'].replace('Snowing + high winds','Snow',inplace=True)
df_cat['Weather_Conditions'].replace('Fog or mist','Fog',inplace=True)


In [ ]:
df_cat['Weather_Conditions'].value_counts()

In [ ]:
df_group = df_cat.groupby(['Accident_Severity', 'Weather_Conditions']).size().reset_index(name='count')

df_pivot = pd.pivot_table(df_group, values='count', 
                     index=['Weather_Conditions'], 
                     columns='Accident_Severity')
# Normalize each day so it is a percentage
df_pivot['Fatal'] = df_pivot['Fatal']/df_pivot['Fatal'].sum()*100.
df_pivot['Serious'] = df_pivot['Serious']/df_pivot['Serious'].sum()*100.
df_pivot['Slight'] = df_pivot['Slight']/df_pivot['Slight'].sum()*100.
df_pivot

In [ ]:
fig  = plt.figure(figsize=(7,7))
ax1 = fig.add_subplot(1,1,1)
sns.heatmap(df_pivot, annot=True,fmt='.3f', linewidths=.3, ax=ax1,  cmap='coolwarm')

fontsize=14
ax1.set_title('Percentage of Accidents by Accident Severity: Weather Conditions',fontsize=fontsize-2)
ax1.set_ylabel('Weather Conditions',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
# ax1.set_xticks(range(len(categories)))
# ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
plt.yticks(va='center')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/weather_cond.png',dpi=300)


Again, no significant trends with accident severity.

### Accident Severity & Vehicle Type

Before we start feature engineering, let's see if we can visualize how vehicle type relates to accident severity using a heatmap:

In [ ]:
df_cat['Vehicle_Type'].value_counts()

The problem here is that there are a lot of categories that could be put together, such as the motorcycles, buses, and goods vehicles into single labels. 
<br><br/>
Let's aggregate similar labels: 

In [ ]:
# Lump motorcycles into same label
df_cat['Vehicle_Type'].replace('Electric motorcycle','Motorcycle',inplace=True)
df_cat['Vehicle_Type'].replace('Motorcycle - unknown cc','Motorcycle',inplace=True)
df_cat['Vehicle_Type'].replace('Motorcycle 125cc and under','Motorcycle',inplace=True)
df_cat['Vehicle_Type'].replace('Motorcycle 50cc and under','Motorcycle',inplace=True)
df_cat['Vehicle_Type'].replace('Motorcycle over 125cc and up to 500cc','Motorcycle',inplace=True)
df_cat['Vehicle_Type'].replace('Motorcycle over 500cc','Motorcycle',inplace=True)
# Buses 
df_cat['Vehicle_Type'].replace('Minibus (8 - 16 passenger seats)','Bus',inplace=True)
df_cat['Vehicle_Type'].replace('Bus or coach (17 or more pass seats)','Bus',inplace=True)
# Goods transportation
df_cat['Vehicle_Type'].replace('Goods 7.5 tonnes mgw and over','Goods',inplace=True)
df_cat['Vehicle_Type'].replace('Goods over 3.5t. and under 7.5t','Goods',inplace=True)
df_cat['Vehicle_Type'].replace('Goods vehicle - unknown weight','Goods',inplace=True)
df_cat['Vehicle_Type'].replace('Van / Goods 3.5 tonnes mgw or under','Goods',inplace=True)
# Lets rename some of these labels to something simpler
df_cat['Vehicle_Type'].replace('Taxi/Private hire car','Taxi',inplace=True)
df_cat['Vehicle_Type'].replace('Pedal cycle','Bike',inplace=True)
df_cat['Vehicle_Type'].replace('Agricultural vehicle','Agriculture',inplace=True)
df_cat['Vehicle_Type'].replace('Mobility scooter','Handicap scooter',inplace=True)
# df_cat['Vehicle_Type'].replace('Ridden horse','Horse',inplace=True)
# df_cat['Vehicle_Type'].replace('Tram','Rail',inplace=True)
# Let's finally remove some less common means of transportation that we have no interest in
df_cat.drop(df_cat[df_cat['Vehicle_Type']=='Handicap scooter'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Vehicle_Type']=='Agriculture'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Vehicle_Type']=='Bus'].index, inplace = True) 
df_cat.drop(df_cat[df_cat['Vehicle_Type']=='Taxi'].index, inplace = True) 


Now let's look at the results as a heatmap:

In [ ]:
df_cat['Vehicle_Type'].value_counts()

In [ ]:
df_group = df_cat.groupby(['Accident_Severity', 'Vehicle_Type']).size().reset_index(name='count')

df_pivot = pd.pivot_table(df_group, values='count', 
                     index=['Vehicle_Type'], 
                     columns='Accident_Severity')
df_pivot['Fatal'] = df_pivot['Fatal']/df_pivot['Fatal'].sum()*100.
df_pivot['Serious'] = df_pivot['Serious']/df_pivot['Serious'].sum()*100.
df_pivot['Slight'] = df_pivot['Slight']/df_pivot['Slight'].sum()*100.
df_pivot

In [ ]:

fig  = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(1,1,1)
sns.heatmap(df_pivot, annot=True,fmt='.3f', linewidths=.3, ax=ax1,  cmap='RdPu')

fontsize=14
ax1.set_title('Vehicle Type \& Accident Severity',fontsize=fontsize)
ax1.set_ylabel('\% of Accidents',fontsize=fontsize)
ax1.set_xlabel('Accident Severity',fontsize=fontsize)
# ax1.set_xticks(range(len(categories)))
# ax1.set_xticklabels(categories)
ax1.tick_params(axis='both', labelsize=fontsize)
plt.yticks(va='center')

plt.tight_layout()

# plt.savefig('/Users/rem/IBM_data_science/9_Capstone/figures/vehicle_type.png',dpi=300)


 Here we can see that Vehicle_Type is a good indicator for accident severity for at least Good and Motorcycle vehicle types, indicated by clear differences between severities.  Cars still dominate the fraction of accidents in each severity category, but 15% more for slight severities than fatal severities.

### Output the Final DataFrame as a CSV File

In [ ]:
df_cat.head(10)

In [ ]:
for c in df_cat.columns:
    print('Column: %s' % c)
    print('-------------------------------------------------')
    print(df_cat[c].value_counts())
    print('-------------------------------------------------')
    print()

In [ ]:
# df_cat.to_csv('uk_accident_data.csv',index=False)